1. day period -- close ma10 upper cross ma20
2. day period -- volume : volume upper cross ma5
3. day period -- volume : ma5 upper cross ma40
4. day period -- volume : ma40 upper cross ma135

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime #, time

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [30]:
stock_list = QA.QA_fetch_stock_list_adv()

In [12]:
stock_list = stock_list.index.tolist()

In [29]:
end = '2018-12-20'
# start = QA.QA_util_get_last_day(end,n=140)
start = '2018-01-01'

In [31]:
stock_list.head()

,code,decimal_point,name,pre_close,sec,sse,volunit
code,,,,,,,
000001,000001,2,平安银行,440.022500,stock_cn,sz,100
000002,000002,2,万 科Ａ,25.299999,stock_cn,sz,100
000004,000004,2,国农科技,16.440001,stock_cn,sz,100
000005,000005,2,世纪星源,3522.000625,stock_cn,sz,100
000006,000006,2,深振业Ａ,5.450000,stock_cn,sz,100


In [50]:
code_list_non_st = stock_list[stock_list.name.apply(lambda x : True if ((x[:2].lower() != 'st') | \
                      (x[:3].lower() != '*st')) else False)].index.tolist()

In [52]:
# code_list_non_st

In [54]:
data = QA.QA_fetch_stock_day_adv(code_list_non_st, start, end)

In [17]:
def buy_triple(dataframe):
    CLOSE = dataframe.close
    VOLUME = dataframe.volume
    
    close_ma10 = QA.MA(CLOSE, 10)
    close_ma20 = QA.MA(CLOSE, 20)
    
    volume_ma5 = QA.MA(VOLUME, 5)
    volume_ma40 = QA.MA(VOLUME, 40)
    volume_ma135 = QA.MA(VOLUME, 135)
    
    var = QA.CROSS(close_ma10, close_ma20) * QA.CROSS(VOLUME, volume_ma5) * \
    QA.CROSS(volume_ma5, volume_ma40) * QA.CROSS(volume_ma40, volume_ma135)
    
    return pd.DataFrame({'buyTriple':var})

In [18]:
ind = data.add_func(buy_triple)

In [19]:
inc = QA.QA_DataStruct_Indicators(ind)

In [28]:
ind[ind.buyTriple == 1]

,,buyTriple
date,code,
2018-12-13,601100,1
